Fonction à importer


In [818]:
# Core libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
# Sklearn functionality
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import export_text
from sklearn import preprocessing
# Keras functionality
import requests
# Ignore warnings
pd.options.mode.chained_assignment = None

Récupération des données

In [819]:
# Import data
data_stat_equipe_2023 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/Statistiques-Ligue-1#stats_squads_standard_for").content)[0]
data_stat_equipe_2022 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2021-2022/Stats-2021-2022-Ligue-1#results2021-2022131_overall").content)[0]
data_stat_equipe_2021 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2020-2021/Stats-2020-2021-Ligue-1#results2020-2021131_overall").content)[0]
data_stat_equipe_2020 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2019-2020/Stats-2019-2020-Ligue-1#results2019-2020131_overall").content)[0]
data_stat_equipe_2019 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2018-2019/Stats-2018-2019-Ligue-1#results2018-2019131_overall").content)[0]
data_matchs_ligue1_2023 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/calendrier/Scores-et-tableaux-Ligue-1#sched_2022-2023_13_1").content)[0]
data_matchs_ligue1_2022 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2021-2022/calendrier/Calendrier-et-resultats-2021-2022-Ligue-1#sched_all").content)[0]
data_matchs_ligue1_2021 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2020-2021/calendrier/Calendrier-et-resultats-2020-2021-Ligue-1#sched_all").content)[0]
data_matchs_ligue1_2020 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2019-2020/calendrier/Calendrier-et-resultats-2019-2020-Ligue-1#sched_2019-2020_13_1").content)[0]
data_matchs_ligue1_2019 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2018-2019/calendrier/Calendrier-et-resultats-2018-2019-Ligue-1#sched_all").content)[0]

Importer des données pour voir calculer le profit

In [820]:
# Récupération des données de pari
data_pari_2023 = pd.read_csv("pari_2023.csv")
data_pari_2022 = pd.read_csv("pari_2022.csv")
data_pari_2021 = pd.read_csv("pari_2021.csv")
data_pari_2020 = pd.read_csv("pari_2020.csv")
data_pari_2019 = pd.read_csv("pari_2019.csv")
# Concaténation des données de pari
data_pari = pd.concat([data_pari_2019,data_pari_2020,data_pari_2021,data_pari_2022,data_pari_2023],axis=0)
# Suppression des données inutiles
pari= data_pari[["B365H","B365D","B365A","Date","HomeTeam","AwayTeam"]]
# Conversion de la date en datetime
pari["Date"] =pd.to_datetime(pari["Date"])

C:\Users\jules\AppData\Local\Temp\ipykernel_28352\4235135274.py:12: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  pari["Date"] =pd.to_datetime(pari["Date"])


Lier les données

In [821]:
# On prend les données de 2019 à 2023
list_concat = [data_matchs_ligue1_2019,data_matchs_ligue1_2020, data_matchs_ligue1_2021, data_matchs_ligue1_2022, data_matchs_ligue1_2023]
matchs_ligue1 = pd.concat(list_concat)

Rajouter données utile 2023

In [822]:
# On copie les données pour ne pas modifier les données d'origine
matchs_ligue1_2023 = data_matchs_ligue1_2023.copy()
# On met le score sous forme de nombre
matchs_ligue1_2023["but_domicile"] = matchs_ligue1_2023["Score"].str[0]
matchs_ligue1_2023["but_exterieur"] = matchs_ligue1_2023["Score"].str[2]

# On met les points en fonction du score
matchs_ligue1_2023["Points_domicile"] = 0
matchs_ligue1_2023["Points_exterieur"] = 0
matchs_ligue1_2023.loc[matchs_ligue1_2023["but_domicile"]>matchs_ligue1_2023["but_exterieur"],"Points_domicile"] =3 
matchs_ligue1_2023.loc[matchs_ligue1_2023["but_domicile"]<matchs_ligue1_2023["but_exterieur"],"Points_exterieur"] =3
matchs_ligue1_2023.loc[matchs_ligue1_2023["but_domicile"]==matchs_ligue1_2023["but_exterieur"],["Points_domicile","Points_exterieur"]] =1
        

Créer les stats équipe pour chaque journée année 2023

In [823]:
# On crée la liste qui contiendra les statistiques de chaque équipe à chaque journée
stat_2023=[]
# On ajoute les équipes en début de saison
stat_2023.append(data_stat_equipe_2023[["Équipe", "MJ", "BM", "BE", "Pts"]])
stat_2023[0][['MJ','BM', 'BE', 'Pts',"Domicile"]] = 0
# On ajoute les statistiques de chaque équipe à chaque journée
for k in range(37):
    # On initialise la journée à l'aide de la journée précédente
    stat_2023.append(stat_2023[k].copy())
    # Le nombre de match joué augmente de 1
    stat_2023[k+1].loc[:, 'MJ'] += 1
    # On s'occupe de chaque équipe
    for i in range(len(stat_2023[k+1])):
        # On récupère les données du match de l'équipe i à la journée k+1
        donnee_match = matchs_ligue1_2023.loc[(matchs_ligue1_2023["Sem."]==k+1) & ((matchs_ligue1_2023["Domicile"]==stat_2023[k+1].loc[i, 'Équipe'])| (matchs_ligue1_2023["Extérieur"]==stat_2023[k+1].loc[i, 'Équipe']))] 
        # On vérifie si l'équipe est à domicile ou à l'extérieur
        if donnee_match.iloc[0,4]==stat_2023[k+1].loc[i, 'Équipe']:
            domicile = 1
        else:
            domicile = 0
        # On note si l'équipe est à domicile ou à l'extérieur au match k+1
        stat_2023[k].loc[i,'Domicile'] = domicile
        # On note la date du match k+1
        stat_2023[k].loc[i,"date"] = donnee_match.iloc[0,2]
        # On vérifie que le match a bien eu lieu
        if type(donnee_match.iloc[0,14]) != str:
            continue
        # On convertit les données en int
        donnee_match.iloc[0,14] = int(donnee_match.iloc[0,14])
        donnee_match.iloc[0,15] = int(donnee_match.iloc[0,15])
        donnee_match.iloc[0,16] = int(donnee_match.iloc[0,16])
        donnee_match.iloc[0,17] = int(donnee_match.iloc[0,17])
        # On met à jour les statistiques de l'équipe i à la journée k+1
        if domicile:
            stat_2023[k+1].loc[i,"Pts"] =(k*stat_2023[k+1].loc[i,"Pts"] + donnee_match.iloc[0,16])/ (k+1)
            stat_2023[k+1].loc[i,"BM"] = (k*stat_2023[k+1].loc[i,"BM"] + donnee_match.iloc[0,14])/ (k+1)
            stat_2023[k+1].loc[i,"BE"] = (k*stat_2023[k+1].loc[i,"BE"] + donnee_match.iloc[0,15])/ (k+1)
        else:
            stat_2023[k+1].loc[i,"Pts"] =(k*stat_2023[k+1].loc[i,"Pts"] + donnee_match.iloc[0,17])/ (k+1)
            stat_2023[k+1].loc[i,"BM"] = (k*stat_2023[k+1].loc[i,"BM"] + donnee_match.iloc[0,15])/ (k+1)
            stat_2023[k+1].loc[i,"BE"] = (k*stat_2023[k+1].loc[i,"BE"] + donnee_match.iloc[0,14])/ (k+1)
    # On trie les équipes par points moyens
    stat_2023[k+1].sort_values(by=['Pts'], ascending=False, inplace=True)
    # On réinitialise l'index
    stat_2023[k+1].reset_index(drop=True, inplace=True)
# On concatène les statistiques de chaque journée    
stats_2023 = pd.concat(stat_2023)


Pareille année 2022

In [824]:
#Clean année 2022
# On copie les données pour ne pas modifier les données d'origine
matchs_ligue1_2022 = data_matchs_ligue1_2022.copy()
# On enlève les matchs de barrages
matchs_ligue1_2022 = matchs_ligue1_2022[matchs_ligue1_2022["Tour"] != "Barrage 1/2 Relégation/Promotion France"]
# On enlève la colonne Tour
matchs_ligue1_2022.drop(columns="Tour", inplace=True)
matchs_ligue1_2022["but_domicile"] = matchs_ligue1_2022["Score"].str[0]
matchs_ligue1_2022["but_exterieur"] = matchs_ligue1_2022["Score"].str[2]
matchs_ligue1_2022["Points_domicile"] = 0
matchs_ligue1_2022["Points_exterieur"] = 0
matchs_ligue1_2022.loc[matchs_ligue1_2022["but_domicile"]>matchs_ligue1_2022["but_exterieur"],"Points_domicile"] =3 
matchs_ligue1_2022.loc[matchs_ligue1_2022["but_domicile"]<matchs_ligue1_2022["but_exterieur"],"Points_exterieur"] =3
matchs_ligue1_2022.loc[matchs_ligue1_2022["but_domicile"]==matchs_ligue1_2022["but_exterieur"],["Points_domicile","Points_exterieur"]] =1
stat_2022=[]
stat_2022.append(data_stat_equipe_2022[["Équipe", "MJ", "BM", "BE", "Pts"]])
stat_2022[0][['MJ','BM', 'BE', 'Pts',"Domicile"]] = 0
for k in range(37):
    stat_2022.append(stat_2022[k].copy())
    stat_2022[k+1].loc[:, 'MJ'] += 1
    for i in range(len(stat_2022[k+1])):
        donnee_match = matchs_ligue1_2022.loc[(matchs_ligue1_2022["Sem."]==k+1) & ((matchs_ligue1_2022["Domicile"]==stat_2022[k+1].loc[i, 'Équipe'])| (matchs_ligue1_2022["Extérieur"]==stat_2022[k+1].loc[i, 'Équipe']))] 
        if donnee_match.iloc[0,4]==stat_2022[k+1].loc[i,'Équipe']:
            domicile = 1
        else:
            domicile = 0
        stat_2022[k].loc[i,'Domicile'] = domicile
        stat_2022[k].loc[i,"date"] = donnee_match.iloc[0,2]
        if type(donnee_match.iloc[0,14]) != str:
            continue
        donnee_match.iloc[0,14] = int(donnee_match.iloc[0,14])
        donnee_match.iloc[0,15] = int(donnee_match.iloc[0,15])
        donnee_match.iloc[0,16] = int(donnee_match.iloc[0,16])
        donnee_match.iloc[0,17] = int(donnee_match.iloc[0,17])
        if domicile:
            stat_2022[k+1].loc[i,"Pts"] =(k*stat_2022[k+1].loc[i,"Pts"] + donnee_match.iloc[0,16])/(k+1)
            stat_2022[k+1].loc[i,"BM"] = (k*stat_2022[k+1].loc[i,"BM"] + donnee_match.iloc[0,14])/ (k+1)
            stat_2022[k+1].loc[i,"BE"] = (k*stat_2022[k+1].loc[i,"BE"] + donnee_match.iloc[0,15])/ (k+1)
        else:
            stat_2022[k+1].loc[i,"Pts"] =(k*stat_2022[k+1].loc[i,"Pts"] + donnee_match.iloc[0,17])/(k+1)
            stat_2022[k+1].loc[i,"BM"] = (k*stat_2022[k+1].loc[i,"BM"] + donnee_match.iloc[0,15])/ (k+1)
            stat_2022[k+1].loc[i,"BE"] = (k*stat_2022[k+1].loc[i,"BE"] + donnee_match.iloc[0,14])/ (k+1)
    stat_2022[k+1].sort_values(by=['Pts'], ascending=False, inplace=True)
    stat_2022[k+1].reset_index(drop=True, inplace=True)
stats_2022 = pd.concat(stat_2022)

Année 2021

In [825]:
#Clean année 2021
# On copie les données pour ne pas modifier les données d'origine
matchs_ligue1_2021 = data_matchs_ligue1_2021.copy()
matchs_ligue1_2021 = matchs_ligue1_2021[matchs_ligue1_2021["Tour"] != "Barrage 1/2 Relégation/Promotion France"]
matchs_ligue1_2021.drop(columns="Tour", inplace=True)
matchs_ligue1_2021["but_domicile"] = matchs_ligue1_2021["Score"].str[0]
matchs_ligue1_2021["but_exterieur"] = matchs_ligue1_2021["Score"].str[2]
matchs_ligue1_2021["Points_domicile"] = 0
matchs_ligue1_2021["Points_exterieur"] = 0
matchs_ligue1_2021.loc[matchs_ligue1_2021["but_domicile"]>matchs_ligue1_2021["but_exterieur"],"Points_domicile"] =3 
matchs_ligue1_2021.loc[matchs_ligue1_2021["but_domicile"]<matchs_ligue1_2021["but_exterieur"],"Points_exterieur"] =3
matchs_ligue1_2021.loc[matchs_ligue1_2021["but_domicile"]==matchs_ligue1_2021["but_exterieur"],["Points_domicile","Points_exterieur"]] =1

stat_2021=[]
stat_2021.append(data_stat_equipe_2021[["Équipe", "MJ", "BM", "BE", "Pts"]])
stat_2021[0][['MJ','BM', 'BE', 'Pts',"Domicile"]] = 0
for k in range(37):
    stat_2021.append(stat_2021[k].copy())
    stat_2021[k+1].loc[:, 'MJ'] += 1
    for i in range(len(stat_2021[k+1])):
        donnee_match = matchs_ligue1_2021.loc[(matchs_ligue1_2021["Sem."]==k+1) & ((matchs_ligue1_2021["Domicile"]==stat_2021[k+1].loc[i, 'Équipe'])| (matchs_ligue1_2021["Extérieur"]==stat_2021[k+1].loc[i, 'Équipe']))] 
        if donnee_match.iloc[0,4]==stat_2021[k+1].loc[i, 'Équipe']:
            domicile = 1
        else:
            domicile = 0
        stat_2021[k].loc[i,'Domicile'] = domicile
        stat_2021[k].loc[i,"date"] = donnee_match.iloc[0,2]
        if type(donnee_match.iloc[0,14]) != str:
            continue
        donnee_match.iloc[0,14] = int(donnee_match.iloc[0,14])
        donnee_match.iloc[0,15] = int(donnee_match.iloc[0,15])
        donnee_match.iloc[0,16] = int(donnee_match.iloc[0,16])
        donnee_match.iloc[0,17] = int(donnee_match.iloc[0,17])
        if domicile:
            stat_2021[k+1].loc[i,"Pts"] =(k*stat_2021[k+1].loc[i,"Pts"] + donnee_match.iloc[0,16])/(k+1)
            stat_2021[k+1].loc[i,"BM"] = (k*stat_2021[k+1].loc[i,"BM"] + donnee_match.iloc[0,14])/ (k+1)
            stat_2021[k+1].loc[i,"BE"] = (k*stat_2021[k+1].loc[i,"BE"] + donnee_match.iloc[0,15])/ (k+1)
        else:
            stat_2021[k+1].loc[i,"Pts"] =(k*stat_2021[k+1].loc[i,"Pts"] + donnee_match.iloc[0,17])/(k+1)
            stat_2021[k+1].loc[i,"BM"] = (k*stat_2021[k+1].loc[i,"BM"] + donnee_match.iloc[0,15])/ (k+1)
            stat_2021[k+1].loc[i,"BE"] = (k*stat_2021[k+1].loc[i,"BE"] + donnee_match.iloc[0,14])/ (k+1)
        stat_2021[k+1].sort_values(by=['Pts'], ascending=False, inplace=True)
        stat_2021[k+1].reset_index(drop=True, inplace=True)
stats_2021 = pd.concat(stat_2021)

Année 2020

In [826]:
#Clean année 2020
# On copie les données pour ne pas modifier les données d'origine
matchs_ligue1_2020 = data_matchs_ligue1_2020.copy()
matchs_ligue1_2020["but_domicile"] = matchs_ligue1_2020["Score"].str[0]
matchs_ligue1_2020["but_exterieur"] = matchs_ligue1_2020["Score"].str[2]
matchs_ligue1_2020["Points_domicile"] = 0
matchs_ligue1_2020["Points_exterieur"] = 0
matchs_ligue1_2020.loc[matchs_ligue1_2020["but_domicile"]>matchs_ligue1_2020["but_exterieur"],"Points_domicile"] =3 
matchs_ligue1_2020.loc[matchs_ligue1_2020["but_domicile"]<matchs_ligue1_2020["but_exterieur"],"Points_exterieur"] =3
matchs_ligue1_2020.loc[matchs_ligue1_2020["but_domicile"]==matchs_ligue1_2020["but_exterieur"],["Points_domicile","Points_exterieur"]] =1

stat_2020=[]
stat_2020.append(data_stat_equipe_2020[["Équipe", "MJ", "BM", "BE", "Pts"]])
stat_2020[0][['MJ','BM', 'BE', 'Pts',"Domicile"]] = 0
for k in range(38):
    stat_2020.append(stat_2020[k].copy())
    stat_2020[k+1].loc[:, 'MJ'] += 1
    for i in range(len(stat_2020[k+1])):
        donnee_match = matchs_ligue1_2020.loc[(matchs_ligue1_2020["Sem."]==k+1) & ((matchs_ligue1_2020["Domicile"]==stat_2020[k+1].loc[i, 'Équipe'])| (matchs_ligue1_2020["Extérieur"]==stat_2020[k+1].loc[i, 'Équipe']))] 
        if donnee_match.iloc[0,4]==stat_2020[k+1].loc[i, 'Équipe']:
            domicile = 1
        else:
            domicile = 0
        stat_2020[k].loc[i,'Domicile'] = domicile
        stat_2020[k].loc[i,"date"] = donnee_match.iloc[0,2]
        if type(donnee_match.iloc[0,14]) != str:
            continue
        donnee_match.iloc[0,14] = int(donnee_match.iloc[0,14])
        donnee_match.iloc[0,15] = int(donnee_match.iloc[0,15])
        donnee_match.iloc[0,16] = int(donnee_match.iloc[0,16])
        donnee_match.iloc[0,17] = int(donnee_match.iloc[0,17])
        if domicile:
            stat_2020[k+1].loc[i,"Pts"] =(k*stat_2020[k+1].loc[i,"Pts"] + donnee_match.iloc[0,16])/(k+1)
            stat_2020[k+1].loc[i,"BM"] = (k*stat_2020[k+1].loc[i,"BM"] + donnee_match.iloc[0,14])/ (k+1)
            stat_2020[k+1].loc[i,"BE"] = (k*stat_2020[k+1].loc[i,"BE"] + donnee_match.iloc[0,15])/ (k+1)
        else:
            stat_2020[k+1].loc[i,"Pts"] =(k*stat_2020[k+1].loc[i,"Pts"] + donnee_match.iloc[0,17])/(k+1)
            stat_2020[k+1].loc[i,"BM"] = (k*stat_2020[k+1].loc[i,"BM"] + donnee_match.iloc[0,15])/ (k+1)
            stat_2020[k+1].loc[i,"BE"] = (k*stat_2020[k+1].loc[i,"BE"] + donnee_match.iloc[0,14])/ (k+1)
        stat_2020[k+1].sort_values(by=['Pts'], ascending=False, inplace=True)
        stat_2020[k+1].reset_index(drop=True, inplace=True)
stats_2020 = pd.concat(stat_2020)

Année 2019

In [827]:
#Clean année 2019
# On copie les données pour ne pas modifier les données d'origine
matchs_ligue1_2019 = data_matchs_ligue1_2019.copy()
matchs_ligue1_2019 = matchs_ligue1_2019[matchs_ligue1_2019["Tour"] != "Barrage 1/2 Relégation/Promotion France"]
matchs_ligue1_2019.drop(columns="Tour", inplace=True)
matchs_ligue1_2019["but_domicile"] = matchs_ligue1_2019["Score"].str[0]
matchs_ligue1_2019["but_exterieur"] = matchs_ligue1_2019["Score"].str[2]
matchs_ligue1_2019["Points_domicile"] = 0
matchs_ligue1_2019["Points_exterieur"] = 0
matchs_ligue1_2019.loc[matchs_ligue1_2019["but_domicile"]>matchs_ligue1_2019["but_exterieur"],"Points_domicile"] =3 
matchs_ligue1_2019.loc[matchs_ligue1_2019["but_domicile"]<matchs_ligue1_2019["but_exterieur"],"Points_exterieur"] =3
matchs_ligue1_2019.loc[matchs_ligue1_2019["but_domicile"]==matchs_ligue1_2019["but_exterieur"],["Points_domicile","Points_exterieur"]] =1

stat_2019=[]
stat_2019.append(data_stat_equipe_2019[["Équipe", "MJ", "BM", "BE", "Pts"]])
stat_2019[0][['MJ','BM', 'BE', 'Pts',"Domicile"]] = 0
for k in range(38):
    stat_2019.append(stat_2019[k].copy())
    stat_2019[k+1].loc[:, 'MJ'] += 1
    for i in range(len(stat_2019[k+1])):
        donnee_match = matchs_ligue1_2019.loc[(matchs_ligue1_2019["Sem."]==k+1) & ((matchs_ligue1_2019["Domicile"]==stat_2019[k+1].loc[i, 'Équipe'])| (matchs_ligue1_2019["Extérieur"]==stat_2019[k+1].loc[i, 'Équipe']))] 
        if donnee_match.iloc[0,4]==stat_2019[k+1].loc[i, 'Équipe']:
            domicile = 1
        else:
            domicile = 0
        stat_2019[k].loc[i,'Domicile'] = domicile
        stat_2019[k+1].loc[i,"date"] = donnee_match.iloc[0,2]
        if type(donnee_match.iloc[0,14]) != str:
            continue
        donnee_match.iloc[0,14] = int(donnee_match.iloc[0,14])
        donnee_match.iloc[0,15] = int(donnee_match.iloc[0,15])
        donnee_match.iloc[0,16] = int(donnee_match.iloc[0,16])
        donnee_match.iloc[0,17] = int(donnee_match.iloc[0,17])
        if domicile:
            stat_2019[k+1].loc[i,"Pts"] =(k*stat_2019[k+1].loc[i,"Pts"] + donnee_match.iloc[0,16])/(k+1)
            stat_2019[k+1].loc[i,"BM"] = (k*stat_2019[k+1].loc[i,"BM"] + donnee_match.iloc[0,14])/ (k+1)
            stat_2019[k+1].loc[i,"BE"] = (k*stat_2019[k+1].loc[i,"BE"] + donnee_match.iloc[0,15])/ (k+1)
        else:
            stat_2019[k+1].loc[i,"Pts"] =(k*stat_2019[k+1].loc[i,"Pts"] + donnee_match.iloc[0,17])/(k+1)
            stat_2019[k+1].loc[i,"BM"] = (k*stat_2019[k+1].loc[i,"BM"] + donnee_match.iloc[0,15])/ (k+1)
            stat_2019[k+1].loc[i,"BE"] = (k*stat_2019[k+1].loc[i,"BE"] + donnee_match.iloc[0,14])/ (k+1)
        stat_2019[k+1].sort_values(by=['Pts'], ascending=False, inplace=True)
        stat_2019[k+1].reset_index(drop=True, inplace=True)
stats_2019 = pd.concat(stat_2019)

Concatener toute les données

In [828]:
# On concatène les stats de chaque année
stats = pd.concat([stats_2019, stats_2020, stats_2021, stats_2022,stats_2023])
# On convertit la date en datetime
stats["date"]= pd.to_datetime(stats["date"])
stats["key"] = stats["Équipe"] + stats["date"].astype(str)
stats.drop_duplicates(subset=["key"], inplace=True)

Nettoyer les données des matchs

In [829]:
# Enlever les colonnes inutiles
matchs_ligue1 = matchs_ligue1[matchs_ligue1["Tour"] != "Barrage 1/2 Relégation/Promotion France"]
matchs_trier= matchs_ligue1[["Date","Domicile","Score","Extérieur","Sem."]]
# Enlever les lignes inutiles
matchs_trier = matchs_trier[matchs_trier["Date"].notna()]
# Mettre les dates au bon format
matchs_trier["Date"] = pd.to_datetime(matchs_trier["Date"])
# Garder les matchs joués
matchs_joue = matchs_trier.dropna(subset=["Score"])
# Remettre les index à 0
matchs_joue.reset_index(drop=True, inplace=True)
# Ajouter les colonnes but_domicile et but_exterieur
matchs_joue["but_domicile"] = matchs_joue["Score"].str[0]
matchs_joue["but_exterieur"] = matchs_joue["Score"].str[2]
# garder les matchs à jouer
matchs_a_jouer = matchs_trier[matchs_trier["Score"].isna()]
# Mettres les buts en type int
matchs_joue["but_domicile"] = matchs_joue["but_domicile"].astype(int)
matchs_joue["but_exterieur"] = matchs_joue["but_exterieur"].astype(int)

#Ajouter les colonnes resultat_domicile et resultat_exterieur
matchs_joue["resultat"] = 0
matchs_joue.loc[matchs_joue["but_domicile"]>matchs_joue["but_exterieur"],"resultat"] = 2
matchs_joue.loc[matchs_joue["but_domicile"] == matchs_joue["but_exterieur"],"resultat"] = 1
matchs_joue.loc[matchs_joue["but_domicile"]<matchs_joue["but_exterieur"],"resultat"] = 0
# Diviser la date en jour, mois et année
matchs_joue["jour"] = matchs_joue["Date"].dt.day
matchs_joue["mois"] = matchs_joue["Date"].dt.month
matchs_joue["annee"] = matchs_joue["Date"].dt.year
# Diviser la date en jour, mois et année pour les matchs à jouer
matchs_a_jouer["jour"] = matchs_a_jouer["Date"].dt.day
matchs_a_jouer["mois"] = matchs_a_jouer["Date"].dt.month
matchs_a_jouer["annee"] = matchs_a_jouer["Date"].dt.year
# Enlever les matchs covid
matchs_a_jouer = matchs_a_jouer.loc[matchs_a_jouer["annee"] == 2023]

Lier les données

In [830]:
# On ajoute les stats des équipes à chaque match
matchs_joue["key1"]= matchs_joue["Domicile"] + matchs_joue["Date"].astype(str)
matchs_joue["key2"]= matchs_joue["Extérieur"] + matchs_joue["Date"].astype(str)
merge1 = pd.merge(matchs_joue, stats,  how='inner', left_on=["key1"], right_on = ["key"])
merge2 = pd.merge(merge1, stats,  how='inner', left_on=["key2"], right_on = ["key",])
merge2.isna().sum()
matchs_a_jouer["key1"]= matchs_a_jouer["Domicile"] + matchs_a_jouer["Date"].astype(str)
matchs_a_jouer["key2"]= matchs_a_jouer["Extérieur"] + matchs_a_jouer["Date"].astype(str)
merge1_a_jouer = pd.merge(matchs_a_jouer, stats,  how='inner', left_on=["key1"], right_on = ["key"])
merge2_a_jouer = pd.merge(merge1_a_jouer, stats,  how='inner', left_on=["key2"], right_on = ["key",])

Lier les données de pari

In [831]:
pari["key"] = pari["HomeTeam"] + pari["Date"].astype(str)
merge3 = pd.merge(merge2, pari,  how='inner', left_on=["Domicile_x","Date"], right_on = ["HomeTeam","Date"])
merge3.isna().mean()

Date             0.000000
Domicile_x       0.000000
Score            0.000000
Extérieur        0.000000
Sem.             0.000000
but_domicile     0.000000
but_exterieur    0.000000
resultat         0.000000
jour             0.000000
mois             0.000000
annee            0.000000
key1             0.000000
key2             0.000000
Équipe_x         0.000000
MJ_x             0.000000
BM_x             0.000000
BE_x             0.000000
Pts_x            0.000000
Domicile_y       0.000000
date_x           0.000000
key_x            0.000000
Équipe_y         0.000000
MJ_y             0.000000
BM_y             0.000000
BE_y             0.000000
Pts_y            0.000000
Domicile         0.000000
date_y           0.000000
key_y            0.000000
B365H            0.001376
B365D            0.001376
B365A            0.001376
HomeTeam         0.000000
AwayTeam         0.000000
key              0.000000
dtype: float64

Trie des données utiles

In [832]:
# On enlève les colonnes inutiles
merge3 = merge3[['Sem.', 'but_domicile',"but_exterieur","jour","mois","annee","Pts_x","BM_x","BE_x","Pts_y","BM_y","BE_y","resultat","B365H","B365D","B365A"]]
matchs_joue_dom_ext = merge3.rename(columns={"Pts_x": "Pts_dom", "BM_x": "BM_dom", "BE_x": "BE_dom","Pts_y": "Pts_ext", "BM_y": "BM_ext", "BE_y": "BE_ext"})
merge2_a_jouer = merge2_a_jouer[['Sem.',"mois","annee","Pts_x","BM_x","BE_x","Pts_y","BM_y","BE_y","Domicile_x","Extérieur"]]
matchs_a_jouer_dom_ext = merge2_a_jouer.rename(columns={"Pts_x": "Pts_dom", "BM_x": "BM_dom", "BE_x": "BE_dom","Pts_y": "Pts_ext", "BM_y": "BM_ext", "BE_y": "BE_ext","Domicile_x":"Domicile"})

Créer les données de test et d'entrainement

On sépare les données pour créer les modèles

In [833]:
# On sépare les données en train et test
X = matchs_joue_dom_ext[['Sem.',"Pts_dom","BM_dom","BE_dom","Pts_ext","BM_ext","BE_ext","mois","annee"]]
y = matchs_joue_dom_ext[['resultat',"B365H","B365D","B365A"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Création du modèle décision tree

In [834]:
# Build a logistic regression model
mode_dt = DecisionTreeClassifier()
mode_dt.fit(X_train,y_train["resultat"])
# Afficher l'arbre de décision
#print("décision_tree pour écart de but \n",export_text(mode1_dt, feature_names=list(X_train)))
# Check the model performance with the training data
predictions_dt_train = mode_dt.predict(X_train)
print("prediction avec les training data:",accuracy_score(y_train["resultat"], predictions_dt_train))
# Check the model performance with the test data
predictions_dt_test = mode_dt.predict(X_test)
print("prediction avec les test data:",accuracy_score(y_test["resultat"], predictions_dt_test))

prediction avec les training data: 0.9960629921259843
prediction avec les test data: 0.4292237442922374


In [840]:
somme_bet = 0
for i in range(len(y_test)):
    if y_test.iloc[i,1] == 0:
        compteur_na += 1
    else:
        if y_test.iloc[i,0] == predictions_dt_test[i]:
            if y_test.iloc[i,0] == 2:
                somme_bet += y_test.iloc[i,1]
            elif y_test.iloc[i,0] == 1:
                somme_bet += y_test.iloc[i,2]
            elif y_test.iloc[i,0] == 0:
                somme_bet += y_test.iloc[i,3]
somme_bet = ((somme_bet-len(y_test))/(len(y_test)))*100
print(somme_bet)

9.789954337899555


Prévision des résultats decision tree

In [836]:
matchs_a_jouer_dom_ext["resultat"] = mode_dt.predict(matchs_a_jouer_dom_ext[['Sem.',"Pts_dom","BM_dom","BE_dom","Pts_ext","BM_ext","BE_ext","mois","annee"]])
resultat_mode_dt = matchs_a_jouer_dom_ext[['Domicile',"Extérieur","resultat"]]
resultat_mode_dt.head(10)

,Domicile,Extérieur,resultat
0,Lille,Lyon,2
1,Auxerre,Rennes,0
2,Brest,Paris S-G,0
3,Clermont Foot,Lens,0
4,Lorient,Troyes,0
5,Ajaccio,Montpellier,2
6,Nantes,Nice,2
7,Angers,Toulouse,2
8,Monaco,Reims,1
9,Marseille,Strasbourg,1


Modele logistic regression nombre de but

In [837]:
# Buill a logistic regression model for the ecart de but
mode_lr = LogisticRegression(solver = 'liblinear', max_iter=1000, multi_class='ovr')
mode_lr.fit(X_train, y_train["resultat"])
# Check the model performance with the training data
predictions_lr_train = mode_lr.predict(X_train)
print("prediction avec les training data:",accuracy_score(y_train["resultat"], predictions_lr_train))
# Check the model performance with the test data
predictions_lr_test = mode_lr.predict(X_test)
print("prediction avec les test data:",accuracy_score(y_test["resultat"], predictions_lr_test))

prediction avec les training data: 0.49015748031496065
prediction avec les test data: 0.4794520547945205


In [849]:
somme_bet = 0
for i in range(len(y_test)):
    if y_test.iloc[i,0] == predictions_lr_test[i]:
        if y_test.iloc[i,0] == 2:
            somme_bet += y_test.iloc[i,1]
        elif y_test.iloc[i,0] == 1:
            somme_bet += y_test.iloc[i,2]
        elif y_test.iloc[i,0] == 0:
            somme_bet += y_test.iloc[i,3]
somme_bet = ((somme_bet-len(y_test))/len(y_test))*100
print(somme_bet)

1.538812785388156


Prévision des résultat logistic regression


In [839]:
matchs_a_jouer_dom_ext["resultat"] = mode_lr.predict(matchs_a_jouer_dom_ext[['Sem.',"Pts_dom","BM_dom","BE_dom","Pts_ext","BM_ext","BE_ext","mois","annee"]])
resultat_mode_lr = matchs_a_jouer_dom_ext[['Domicile',"Extérieur","resultat"]]
resultat_mode_lr.head(10)

,Domicile,Extérieur,resultat
0,Lille,Lyon,2
1,Auxerre,Rennes,0
2,Brest,Paris S-G,0
3,Clermont Foot,Lens,0
4,Lorient,Troyes,2
5,Ajaccio,Montpellier,0
6,Nantes,Nice,0
7,Angers,Toulouse,0
8,Monaco,Reims,2
9,Marseille,Strasbourg,2
